In [ ]:
# !pip freeze > requirements.txt

In [ ]:
from google.colab import drive

#讀取雲端資料夾
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 載入jupyter網頁互動控制套件

!pip install -q ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display

from huggingface_hub import login, logout, hf_hub_download

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Total 607 (delta 0), reused 0 (delta 0), pack-reused 607
Receiving objects: 100% (607/607), 27.84 MiB | 8.88 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [ ]:
# datasets載入資料 loralib使用LORA sentencepiece分詞
!pip install -q datasets loralib sentencepiece

!pip uninstall transformers
!pip install -q git+https://github.com/huggingface/transformers.git

!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.4 MB/s

In [ ]:
# cuda的quatization model專用套件包含優化函數、正規化、位置嵌入等功能
!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
# LlamaForCausalLM載入不同模型權重 載入不同模型tokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer



# EOS => end of seq 輸入一個位置編碼代表單次輸出輸入的最後一個字已經結束
# 載入這個模型的tokneizer
# 這裡要換個分詞器 因為語言不同用來分詞的詞庫也會不同
tokenizer = LlamaTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf", add_eos_token=True)
# 分樣本function
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

data = load_dataset("json", data_files="/content/drive/MyDrive/nlp/data/alpaca-tw_en-align.json")


def generate_prompt(data_point):
    # 檢查樣本instruction值是否為空
    if data_point["instruction"]:
        return f"""下方是一個關於任務的指令，以及一個提供與任務相關之資訊的輸入。請撰寫一個能適當地完成該任務指令需求的回覆。

### 指令:
{data_point["instruction"]}

### 輸入:
{data_point["input"]}

### 回覆:
{data_point["output"]}"""
    else:
        return f"""下方是一個關於任務的指令。請撰寫一個能適當地完成該任務指令需求的回覆。

### 輸入:
{data_point["instruction"]}

### 回覆:
{data_point["output"]}"""


# 對每一個樣本生成一個"prompt"並把promt的value轉成token
data = data.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/104004 [00:00<?, ? examples/s]

# Fine tuning

In [ ]:
import os

#選擇要使用的gpu 若不設定就是使用全部可用資源
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
# auto系列套件可以自動辨認參數指定的模型架構來提供相應的訓練資料
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM

# 載入lora和quatizaiton套件
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

In [ ]:
# RTX4090顯卡的訓練超參數設置

# 單個GPU在梯度更新一次計算的樣本數大小，若有提供GRADIENT_ACCUMULATION_STEPS和BATCH_SIZE則可忽略此參數。
MICRO_BATCH_SIZE = 4  # change to 4 for RTX3090
# 一次
BATCH_SIZE = 128
# 在一次梯度更新之前需要累積幾個micr_batch_size的梯度計算資料
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # paper uses 3
LEARNING_RATE = 3e-4  # from the original paper
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [ ]:
# 載入quatizaiton後的hf模型
model = LlamaForCausalLM.from_pretrained(
    "baffo32/decapoda-research-llama-7B-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LlamaTokenizer.from_pretrained(
    "baffo32/decapoda-research-llama-7B-hf", add_eos_token=True
)
# 準備8bit模型
model = prepare_model_for_int8_training(model)

config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00002-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00003-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

...

pytorch_model-00033-of-00033.bin:   0%|          | 0.00/524M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
# lora設定
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
# 載入lora設定到模型
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", data_files="/content/drive/MyDrive/nlp/data/alpaca-tw_en-align.json")

In [ ]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""下方是一個關於任務的指令，以及一個提供與任務相關之資訊的輸入。請撰寫一個能適當地完成該任務指令需求的回覆。

### 指令:
{data_point["instruction"]}

### 輸入:
{data_point["input"]}

### 回覆:
{data_point["output"]}"""
    else:
        return f"""下方是一個關於任務的指令。請撰寫一個能適當地完成該任務指令需求的回覆。

### 輸入:
{data_point["instruction"]}

### 回覆:
{data_point["output"]}"""



# 把資料集切割成同樣長度並串聯，再用tokenizer轉成向量
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
)
# fp16半精度福點數 相較fp32儲存量減少一半
# logging steps 顯示每步的損失
# 每幾步儲存training logs
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="/content/drive/MyDrive/nlp/training log1/",
        save_total_limit=162,
        save_steps=10
    ),
    # 準備訓練樣本的套件 可以對樣本做padding和對模型做masked label traning
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
# 模型在回覆問題時是否緩存上一個問題的計算內容
model.config.use_cache = False
# 載入上一個訓練的checkpoint繼續訓練
trainer.train(resume_from_checkpoint= False)


model.save_pretrained("/content/drive/MyDrive/nlp")

Map:   0%|          | 0/104004 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.890200


KeyboardInterrupt: 